In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
loader = UnstructuredPDFLoader("../data/pdf_data/AI-Practitioner-Handbook.pdf")
#loader = OnlinePDFLoader("https://epoch.aisingapore.org/wp-content/uploads/2023/03/AI-Practitioner-Handbook-20230324.pdf")

data = loader.load()
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your document')

#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

print (f'Now you have {len(texts)} documents')

In [ ]:
OPENAI_API_KEY = 'xx-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

In [ ]:
import tiktoken
# create a GPT-4 encoder instance
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

total_word_count = sum(len(doc.page_content.split()) for doc in texts)
total_token_count = sum(len(enc.encode(doc.page_content)) for doc in texts)

print(f"\nTotal word count: {total_word_count}")
print(f"\nEstimated tokens: {total_token_count}")
print(f"\nEstimated cost of embedding: ${total_token_count * 0.0004 / 1000}")

In [ ]:
# embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
# vector_store = FAISS.from_documents(texts, embeddings)

# from IPython.display import display, Markdown

# search_result = vector_store.similarity_search_with_score("who is this pdf for?")
# search_result

# line_separator = "\n"# {line_separator}Source: {r[0].metadata['source']}{line_separator}Score:{r[1]}{line_separator}
# display(Markdown(f"""
# ## Search results:{line_separator}
# {line_separator.join([
#   f'''
#   ### Source:{line_separator}{r[0].metadata['source']}{line_separator}
#   #### Score:{line_separator}{r[1]}{line_separator}
#   #### Content:{line_separator}{r[0].page_content}{line_separator}
#   '''
#   for r in search_result
# ])}
# """))

# vector_store.save_local("C:/Users/meldr/aiap/langchain-question-answer/data/vector_data/")

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

system_template="""Use the following pieces of context to answer the users question.
Take note of the sources and include them in the answer in the format: "SOURCES: source1 source2", use "SOURCES" in capital letters regardless of the number of sources.
If you don't know the answer, just say that "I don't know", don't try to make up an answer.
----------------
{summaries}"""
messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]
prompt = ChatPromptTemplate.from_messages(messages)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

In [ ]:
import os

path = "C:/Users/meldr/aiap/langchain-question-answer/data/vector_data/"
if os.path.exists(path):
  vector_store = FAISS.load_local(
      path,
      OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
  )
else:
  print(f"Missing files. Upload index.faiss and index.pkl files to {path} directory first")

In [ ]:
chain_type_kwargs = {"prompt": prompt}
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)  # Modify model_name if you have access to GPT-4
chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
from IPython.display import display, Markdown

def print_result(result):
    output_text = f"""### Question: 
    {query}
    ### Answer: 
    {result['answer']}
    ### Sources: 
    {result['sources']}
    ### All relevant sources:
    {' '.join(list(set([doc.metadata['source'] for doc in result['source_documents']])))}
    """
    display(Markdown(output_text))

In [ ]:
query = "What are the different robustness testing tools?"
# result = chain(query)
print_result(result)
# result